In [46]:
import os
import tensorflow as tf
import numpy as np
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, concatenate


In [111]:
def parse_tfrecord(example):
    features = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'src_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64)
    }
    example = tf.io.parse_single_example(example, features)
    # 이미지 경로 수정
    original_path = tf.strings.join([example['src_path'], tf.constant('/'), example['Raw_data_ID']])
    replacement = 'labelingdata/dog/skin/camera/symp/A1'
    new_path = tf.strings.regex_replace(original_path, '라벨링데이터/반려견/피부/일반카메라/유증상/A1_구진_플라크', replacement)
    # 좌표를 리스트로 변환
    x_coords = tf.sparse.to_dense(example['x_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    y_coords = tf.sparse.to_dense(example['y_coords'], default_value=-1)  # 빈 좌표는 -1로 처리
    # 좌표 튜플로 반환
    coords_pairs = (x_coords, y_coords)
    return new_path, example['lesions'], coords_pairs


def load_dataset(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord)
    return dataset



In [118]:
# TFRecord 파일 경로
tfrecord_file = './val_symptoms_A11.tfrecord'
# 데이터셋 로드
dataset = load_dataset(tfrecord_file)
# 데이터셋 샘플 확인
for example in dataset.take(1):
    image_path, lesions, coords_pairs = example
    print("Image Path:", image_path.numpy())
    print("Lesion:", lesions.numpy())
    x_coords, y_coords = coords_pairs
    print("X Coords:", x_coords.numpy())
    print("Y Coords:", y_coords.numpy())



import os
from PIL import Image, ImageDraw

def crop_and_save_image(image_path, x_coords, y_coords, save_dir):
    # 이미지 로드
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # 좌표를 연결하여 이미지 자르기
    for i in range(len(x_coords) - 1):
        # 현재 점과 다음 점을 연결하여 crop
        x1, y1 = x_coords[i], y_coords[i]
        x2, y2 = x_coords[i+1], y_coords[i+1]
        print(f"Cropping image with coordinates: ({x1}, {y1}) to ({x2}, {y2})")
        
        # 이미지 crop
        cropped_image = image.crop((x1, y1, x2, y2))
        
        # 자르기 영역을 이미지에 표시
        draw.rectangle([(x1, y1), (x2, y2)], outline="red")
        
        # 저장할 파일 경로 생성
        save_path = os.path.join(save_dir, f"cropped_image_{i}.jpg")
        
        # 이미지 저장
        cropped_image.save(save_path)
    
    # 원본 이미지에 자르기 영역 표시된 이미지 저장
    image.save(os.path.join(save_dir, "annotated_image.jpg"))

# 저장할 디렉토리 생성
save_dir = './cropped_images'
os.makedirs(save_dir, exist_ok=True)

# 데이터셋 순회하며 이미지 자르고 저장
for example in dataset.take(1):
    image_path, _, coords_pairs = example
    x_coords, y_coords = coords_pairs
    image_path = './labelingdata/dog/skin/camera/symp/A1/IMG_D_A1_034851.jpg'
    crop_and_save_image(image_path, x_coords.numpy(), y_coords.numpy(), save_dir)
    
print("이미지가 성공적으로 자르고 저장되었습니다.")




Image Path: b'/labelingdata/dog/skin/camera/symp/A1/IMG_D_A1_034851.jpg'
Lesion: b'A1'
X Coords: [ 987  958  943  959 1008 1029 1037 1027 1026 1025  987]
Y Coords: [488 492 458 428 420 431 459 479 479 478 488]
Cropping image with coordinates: (987, 488) to (958, 492)


ValueError: Coordinate 'right' is less than 'left'

---

In [127]:
import tensorflow as tf

def parse_tfrecord(example):
    features = {
        'Raw_data_ID': tf.io.FixedLenFeature([], tf.string),
        'breed': tf.io.FixedLenFeature([], tf.string),
        'age': tf.io.FixedLenFeature([], tf.int64),
        'gender': tf.io.FixedLenFeature([], tf.string),
        'species': tf.io.FixedLenFeature([], tf.string),
        'lesions': tf.io.FixedLenFeature([], tf.string),
        'path': tf.io.FixedLenFeature([], tf.string),
        'src_path': tf.io.FixedLenFeature([], tf.string),
        'x_coords': tf.io.VarLenFeature(tf.int64),
        'y_coords': tf.io.VarLenFeature(tf.int64),
    }
    example = tf.io.parse_single_example(example, features)
    
    # 이미지 경로 수정
    original_path = tf.strings.join([example['src_path'], tf.constant('/'), example['Raw_data_ID']])
    replacement = 'labelingdata/dog/skin/camera/symp/A1'
    new_path = tf.strings.regex_replace(original_path, '라벨링데이터/반려견/피부/일반카메라/유증상/A1_구진_플라크', replacement)
    
    # 좌표를 리스트로 변환
    x_coords = tf.sparse.to_dense(example['x_coords'])
    y_coords = tf.sparse.to_dense(example['y_coords'])
    
    return new_path, example['lesions'], (x_coords, y_coords)

def load_dataset(tfrecord_file):
    dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = dataset.map(parse_tfrecord)
    return dataset


In [137]:
import os
import numpy as np
from PIL import Image, ImageDraw

def crop_and_save_image(image_path, coords_pairs, save_dir, resize_width=None, resize_height=None):
    x_coords, y_coords = coords_pairs
    # 이미지 로드
    original_image = Image.open(image_path)
    draw = ImageDraw.Draw(original_image)
    
    # 좌표값을 원본 이미지에 그리기
    for i in range(len(x_coords)):
        x, y = x_coords[i], y_coords[i]
        draw.text((x, y), f"{i}", fill="red")
    
    # 선으로 좌표 연결하기
    draw.line(list(zip(x_coords, y_coords)), fill="red", width=2)
    
    # 다각형 영역 생성
    mask = Image.new("L", original_image.size, 0)
    draw = ImageDraw.Draw(mask)
    polygon_coords = list(zip(x_coords, y_coords))
    draw.polygon(polygon_coords, outline=1, fill=1)
    
    # 마스크를 이용하여 이미지 자르기
    masked_image = Image.new("RGB", original_image.size)
    masked_image.paste(original_image, mask=mask)
    
    # 이미지 크기 조정
    if resize_width is not None and resize_height is not None:
        masked_image = masked_image.resize((resize_width, resize_height))
    
    # 저장할 파일 경로 생성
    filename = os.path.basename(image_path)
    save_path = os.path.join(save_dir, filename)
    masked_image.save(save_path)
    print(f"이미지가 성공적으로 자르고 저장되었습니다. 저장된 경로: {save_path}")
    
    # 이미지 출력
    original_image.show()

# 저장할 디렉토리 생성
save_dir = './cropped_images'
os.makedirs(save_dir, exist_ok=True)

# 데이터셋 순회하며 이미지 자르고 저장
for example in dataset.take(3):
    image_path, _, coords_pairs = example
    print(image_path)
    image_path = '.\labelingdata\dog\skin\camera\symp\A1\IMG_D_A1_034853.jpg'

    print(image_path)
    print(coords_pairs)
    crop_and_save_image(image_path, coords_pairs, save_dir, resize_width=200, resize_height=200)


tf.Tensor(b'/labelingdata/dog/skin/camera/symp/A1/IMG_D_A1_034851.jpg', shape=(), dtype=string)
.\labelingdata\dog\skin\camera\symp\A1\IMG_D_A1_034853.jpg
(<tf.Tensor: shape=(11,), dtype=int64, numpy=
array([ 987,  958,  943,  959, 1008, 1029, 1037, 1027, 1026, 1025,  987],
      dtype=int64)>, <tf.Tensor: shape=(11,), dtype=int64, numpy=array([488, 492, 458, 428, 420, 431, 459, 479, 479, 478, 488], dtype=int64)>)
이미지가 성공적으로 자르고 저장되었습니다. 저장된 경로: ./cropped_images\IMG_D_A1_034853.jpg
tf.Tensor(b'/labelingdata/dog/skin/camera/symp/A1/IMG_D_A1_034852.jpg', shape=(), dtype=string)
.\labelingdata\dog\skin\camera\symp\A1\IMG_D_A1_034853.jpg
(<tf.Tensor: shape=(9,), dtype=int64, numpy=array([878, 826, 831, 870, 922, 926, 920, 917, 878], dtype=int64)>, <tf.Tensor: shape=(9,), dtype=int64, numpy=array([613, 611, 583, 556, 553, 567, 595, 599, 613], dtype=int64)>)
이미지가 성공적으로 자르고 저장되었습니다. 저장된 경로: ./cropped_images\IMG_D_A1_034853.jpg
tf.Tensor(b'/labelingdata/dog/skin/camera/symp/A1/IMG_D_A1_034853.